In [1]:
%matplotlib inline

import os
import pandas as pd 
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib as mpl
from sklearn.metrics import auc
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier



mpl.rc("figure", figsize=(45,30))
mpl.rcParams.update({'font.size':35})

pd.set_option('display.max_columns', 500)

In [2]:
df = pd.read_csv("../data/train/train.csv")
df.head(10)

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0
5,11,11,795,793,0
6,14,9,444,590,0
7,15,29,538,368,0
8,17,30,857,523,0
9,19,2,559,679,0


In [3]:
campaign_data = pd.read_csv("../data/train/campaign_data.csv")
campaign_data.head(10)

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13
5,22,X,16/09/13,18/10/13
6,18,X,10/08/13,04/10/13
7,19,Y,26/08/13,27/09/13
8,17,Y,29/07/13,30/08/13
9,16,Y,15/07/13,16/08/13


In [4]:
coupon_item_mapping = pd.read_csv("../data/train/coupon_item_mapping.csv")
coupon_item_mapping.head(10)

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77
5,520,77
6,529,77
7,524,77
8,522,81
9,518,81


In [5]:
coupon_item_mapping[coupon_item_mapping.coupon_id == 105]

,coupon_id,item_id
0,105,37
605,105,2360
1957,105,9944
2492,105,13208
3130,105,17451
4232,105,24554
5119,105,30259
5451,105,32173
5976,105,35416
6794,105,39723


One to many mapping

In [6]:
item_data = pd.read_csv("../data/train/item_data.csv")
item_data.head(10)

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery
5,6,56,Local,Grocery
6,7,56,Local,Pharmaceutical
7,8,56,Local,Bakery
8,9,11,Local,Grocery
9,10,56,Local,Grocery


In [7]:
customer_demo = pd.read_csv("../data/train/customer_demographics.csv")
customer_demo.head(10)

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5
5,11,70+,Single,0,2,NaN,1
6,12,46-55,Married,0,2,NaN,7
7,13,36-45,Single,0,1,NaN,2
8,14,26-35,Married,1,2,NaN,6
9,15,46-55,Married,0,2,NaN,6


In [8]:
customer_transaction_data = pd.read_csv("../data/train/customer_transaction_data.csv")
customer_transaction_data

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0
...,...,...,...,...,...,...,...
1324561,2013-06-30,1129,2777,1,284.60,-71.24,0.0
1324562,2013-06-30,1129,2953,4,42.74,-28.50,0.0
1324563,2013-06-30,1129,2971,6,64.12,-42.74,0.0
1324564,2013-06-30,1129,46984,1,95.82,0.00,0.0


In [9]:
customer_transaction_data.shape

(1324566, 7)

Loading test data to confirm if: <br>
1) all customer_ids in test are represented in customer_demographics<br>
2) all customer_ids in test are represented in customer_transaction_data<br>
3) all campaign_ids are represented in campaign_data<br>

In [10]:
test = pd.read_csv("../data/test.csv")
test.head(10)

,id,campaign_id,coupon_id,customer_id
0,3,22,869,967
1,4,20,389,1566
2,5,22,981,510
3,8,25,1069,361
4,10,17,498,811
5,12,25,44,1498
6,13,18,713,682
7,16,22,1079,1186
8,18,18,199,1023
9,31,22,583,315


In [11]:
test.shape

(50226, 4)

In [12]:
(sum((test.customer_id).isin(customer_demo.customer_id))/len((test.customer_id).isin(customer_demo.customer_id)))*100

61.59956994385378

That's only 61% customers present in customer_demo table

In [13]:
len(test.customer_id.unique())

1250

In [14]:
len(customer_demo.customer_id.unique())

760

In [15]:
len(df.customer_id.unique())

1428

So the customer demographics are not available for all customers. Checking in transaction table.

In [16]:
(sum((test.customer_id).isin(customer_transaction_data.customer_id))/len((test.customer_id).isin(customer_transaction_data.customer_id)))*100

100.0

In [17]:
(sum((df.customer_id).isin(customer_transaction_data.customer_id))/len((df.customer_id).isin(customer_transaction_data.customer_id)))*100

100.0

That's fine. So the transaction data for all the customers is present. These features have to be focussed upon as the demographic features will anyway come out to be sparse.

In [18]:
(sum((test.campaign_id).isin(campaign_data.campaign_id))/len((test.campaign_id).isin(campaign_data.campaign_id)))*100

100.0

In [19]:
(sum((df.campaign_id).isin(campaign_data.campaign_id))/len((df.campaign_id).isin(campaign_data.campaign_id)))*100

100.0

Even this is always present.

## Creating a merge

Merging train and test data with a flag is_test

In [20]:
df['is_test'] =0
train = df.copy()
test['is_test'] =1

df = test.append(df, sort=True, ignore_index=True)

In [21]:
train

,id,campaign_id,coupon_id,customer_id,redemption_status,is_test
0,1,13,27,1053,0,0
1,2,13,116,48,0,0
2,6,9,635,205,0,0
3,7,13,644,1050,0,0
4,9,8,1017,1489,0,0
...,...,...,...,...,...,...
78364,128587,8,71,1523,0,0
78365,128589,30,547,937,0,0
78366,128590,8,754,1004,0,0
78367,128592,13,134,71,0,0


In [22]:
df

,campaign_id,coupon_id,customer_id,id,is_test,redemption_status
0,22,869,967,3,1,NaN
1,20,389,1566,4,1,NaN
2,22,981,510,5,1,NaN
3,25,1069,361,8,1,NaN
4,17,498,811,10,1,NaN
...,...,...,...,...,...,...
128590,8,71,1523,128587,0,0.0
128591,30,547,937,128589,0,0.0
128592,8,754,1004,128590,0,0.0
128593,13,134,71,128592,0,0.0


Merging campaign_data and train

In [23]:
merged_df_and_campaign_data = pd.merge(df, campaign_data, left_on='campaign_id', right_on='campaign_id', how='left',
         validate='many_to_one')
merged_df_and_campaign_data

,campaign_id,coupon_id,customer_id,id,is_test,redemption_status,campaign_type,start_date,end_date
0,22,869,967,3,1,NaN,X,16/09/13,18/10/13
1,20,389,1566,4,1,NaN,Y,07/09/13,16/11/13
2,22,981,510,5,1,NaN,X,16/09/13,18/10/13
3,25,1069,361,8,1,NaN,Y,21/10/13,22/11/13
4,17,498,811,10,1,NaN,Y,29/07/13,30/08/13
...,...,...,...,...,...,...,...,...,...
128590,8,71,1523,128587,0,0.0,X,16/02/13,05/04/13
128591,30,547,937,128589,0,0.0,X,19/11/12,04/01/13
128592,8,754,1004,128590,0,0.0,X,16/02/13,05/04/13
128593,13,134,71,128592,0,0.0,X,19/05/13,05/07/13


Merging merged_df_and_campaign_data with coupon_item_mapping with a list of items

In [24]:
coupon_item_mapping['item_id'] = coupon_item_mapping['item_id'].apply(str)
coupon_item_list_mapping = coupon_item_mapping.groupby('coupon_id')['item_id'].apply(lambda x: ','.join(set(x.dropna()))).reset_index()
coupon_item_list_mapping.columns = ['coupon_id', 'item_ids']

coupon_item_list_mapping

,coupon_id,item_ids
0,1,"43986,44868,59108,4140,58940,17091,44112,43951..."
1,2,"12901,2581"
2,3,"59234,59223,58944,59269,59346,58946,58964,5924..."
3,4,"51731,43145,30247,51244,51649,51211,36772,6135..."
4,5,"46144,57016,44994,46006,53496,57079,57118"
...,...,...
1111,1112,"33953,70385,16940,42218,10777,18953"
1112,1113,"24013,21200,35651,20584,55124,15496,38531,5512..."
1113,1114,"14042,31569,30911"
1114,1115,"27865,7765,3413,39245,18175,10123,14657"


In [25]:
len(merged_df_and_campaign_data.coupon_id.unique())

1116

In [26]:
merged_data = pd.merge(merged_df_and_campaign_data, coupon_item_list_mapping, left_on='coupon_id', right_on='coupon_id',
         how='left', validate='many_to_one')
merged_data

,campaign_id,coupon_id,customer_id,id,is_test,redemption_status,campaign_type,start_date,end_date,item_ids
0,22,869,967,3,1,NaN,X,16/09/13,18/10/13,"1124,1185,39835,46911,3514,1033,45271,56179,40..."
1,20,389,1566,4,1,NaN,Y,07/09/13,16/11/13,"47705,45063,19431,56517,1479,68534,56523,1144,..."
2,22,981,510,5,1,NaN,X,16/09/13,18/10/13,"1690,7061,58004,22631,6577,2398,41769,21773,33..."
3,25,1069,361,8,1,NaN,Y,21/10/13,22/11/13,"10284,70009,57994,11281,44510,35356,29737,6994..."
4,17,498,811,10,1,NaN,Y,29/07/13,30/08/13,"45021,54805,14259,37463,48668,16481,4078,19852..."
...,...,...,...,...,...,...,...,...,...,...
128590,8,71,1523,128587,0,0.0,X,16/02/13,05/04/13,"17029,1657,30155,12762"
128591,30,547,937,128589,0,0.0,X,19/11/12,04/01/13,"2596,18777,53061,42386,43044,3318,39336"
128592,8,754,1004,128590,0,0.0,X,16/02/13,05/04/13,"70460,70534,9060,27164,58588,20849,5608,24948,..."
128593,13,134,71,128592,0,0.0,X,19/05/13,05/07/13,"62219,1357,38776,41148,32563,62457,4047"


Now merging the dfs item_data and customer_transaction_data we'll create some new features.

In [27]:
customer_transaction_data = pd.merge(customer_transaction_data, item_data, left_on='item_id', right_on='item_id',
         how='left', validate='many_to_one')
customer_transaction_data

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,524,Established,Grocery
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,524,Established,Grocery
...,...,...,...,...,...,...,...,...,...,...
1324561,2013-06-30,1129,2777,1,284.60,-71.24,0.0,2074,Established,Grocery
1324562,2013-06-30,1129,2953,4,42.74,-28.50,0.0,56,Local,Grocery
1324563,2013-06-30,1129,2971,6,64.12,-42.74,0.0,56,Local,Grocery
1324564,2013-06-30,1129,46984,1,95.82,0.00,0.0,56,Local,Grocery


In [28]:
##This will be used to create a customer to item mapping
customer_transaction_data_on_coupons = customer_transaction_data[customer_transaction_data['coupon_discount'] <0]

#This will be used to derive features about customers purchase history
customer_transaction_data_without_coupons =customer_transaction_data[customer_transaction_data['coupon_discount'] ==0]

customer_transaction_data_on_coupons

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
88,2012-01-02,464,5525,1,106.50,-35.62,-35.62,984,Established,Packaged Meat
93,2012-01-02,464,8145,1,39.18,-24.58,-14.25,1709,Established,Grocery
104,2012-01-02,464,16381,1,48.80,-14.96,-35.62,56,Local,Grocery
107,2012-01-02,464,17861,3,75.51,-32.06,-26.71,124,Established,Grocery
112,2012-01-02,464,19583,2,124.67,-45.59,-35.62,1124,Established,Grocery
...,...,...,...,...,...,...,...,...,...,...
1324282,2013-07-03,384,12071,1,53.43,-26.71,-17.81,1105,Established,Grocery
1324285,2013-07-03,384,22269,1,53.43,-26.71,-17.81,1105,Established,Grocery
1324287,2013-07-03,384,33382,2,106.86,-53.43,-35.62,1105,Established,Grocery
1324403,2013-07-03,1303,69686,1,78.36,0.00,-35.62,522,Established,Grocery


## Saving all merged dfs

In [29]:
merged_data.to_csv("../data/train/merged_data.csv" ,index=False)
customer_transaction_data_on_coupons.to_csv("../data/train/customer_transaction_data_on_coupons.csv" ,index=False)
customer_transaction_data_without_coupons.to_csv("../data/train/customer_transaction_data_without_coupons.csv" ,index=False)

## Making a baseline submission

In [30]:

labelEncoder = LabelEncoder()

merged_data['campaign_type'] = labelEncoder.fit_transform(merged_data['campaign_type'])
merged_data['item_ids'] = labelEncoder.fit_transform(merged_data['item_ids'])
merged_data

,campaign_id,coupon_id,customer_id,id,is_test,redemption_status,campaign_type,start_date,end_date,item_ids
0,22,869,967,3,1,NaN,0,16/09/13,18/10/13,22
1,20,389,1566,4,1,NaN,1,07/09/13,16/11/13,579
2,22,981,510,5,1,NaN,0,16/09/13,18/10/13,105
3,25,1069,361,8,1,NaN,1,21/10/13,22/11/13,8
4,17,498,811,10,1,NaN,1,29/07/13,30/08/13,539
...,...,...,...,...,...,...,...,...,...,...
128590,8,71,1523,128587,0,0.0,0,16/02/13,05/04/13,107
128591,30,547,937,128589,0,0.0,0,19/11/12,04/01/13,228
128592,8,754,1004,128590,0,0.0,0,16/02/13,05/04/13,941
128593,13,134,71,128592,0,0.0,0,19/05/13,05/07/13,821


Re-splitting train and test

In [31]:
train = merged_data[merged_data.is_test ==0]
test = merged_data[merged_data.is_test ==1]
test

,campaign_id,coupon_id,customer_id,id,is_test,redemption_status,campaign_type,start_date,end_date,item_ids
0,22,869,967,3,1,NaN,0,16/09/13,18/10/13,22
1,20,389,1566,4,1,NaN,1,07/09/13,16/11/13,579
2,22,981,510,5,1,NaN,0,16/09/13,18/10/13,105
3,25,1069,361,8,1,NaN,1,21/10/13,22/11/13,8
4,17,498,811,10,1,NaN,1,29/07/13,30/08/13,539
...,...,...,...,...,...,...,...,...,...,...
50221,20,843,501,128584,1,NaN,1,07/09/13,16/11/13,884
50222,25,415,481,128588,1,NaN,1,21/10/13,22/11/13,684
50223,25,596,1336,128591,1,NaN,1,21/10/13,22/11/13,358
50224,22,518,748,128593,1,NaN,0,16/09/13,18/10/13,350


In [32]:
features = ['campaign_id', 'coupon_id', 'campaign_type', 'item_ids']

In [33]:

# Create XGB Classifier object
baseline_model = XGBClassifier(objective = "binary:logistic")

# Fit model
baseline_model.fit(train[features], train['redemption_status'], eval_metric='auc')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [34]:
# Predictions
y_train_preds = baseline_model.predict(train[features])
y_test_preds = baseline_model.predict(test[features])



auc(y_train_preds, train['redemption_status'])

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.0

The model learned nothing basically nothing. Let's make the baseline prediction.

In [35]:
test['redemption_status'] =y_test_preds
submission = test.loc[:,['id', 'redemption_status']]
submission.to_csv("../data/submissions/first_submission.csv" ,index=False)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
